In [1]:
from lstm_functions import *
from lost_functions import *
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
import matplotlib.pyplot as plt

# Use Apple Stock for testing purposes only
This is so that LSTM do not take 2 hour to run  
This notebook is mainly for testing of custom loss function

In [2]:
xls = pd.ExcelFile('data/data_for_testing.xlsx')
apple = pd.read_excel(xls, sheet_name="AAPL").set_index("Date").resample("M").last().reset_index()
all_data = {"AAPL": apple[["Open", "High", "Low", "Close", "Adj Close", "Volume"]]}

In [3]:
final_importance_values = {}
final_predictions = {}
# 30 is not a good number of batches, but it's a start for testing
# 60 is a good number of batches, but it takes a long time to train
time_steps = 1
features = 6

In [4]:
best_loss_value = float('inf')  # Initialize with a high value
best_loss_function = None

evaluation_results = {}  # Store evaluation results for each loss function

In [1]:
for loss_func, loss_name in zip(loss_functions, loss_names):
    print("loss_names", loss_name)
    # Define the model
    lstm_model = LstmBuilder(time_step=time_steps, loss=loss_func, batch_size=4)
    model = lstm_model.create_stateful_model(features=features)
    scaler = MinMaxScaler()
    normalized_data = scaler.fit_transform(all_data["AAPL"])
    X, y = lstm_model.create_sequences(normalized_data)
    X_train, X_test, y_train, y_test = lstm_model.split_stateful_data(X,y)

    # Train the model
    model.fit(X_train, y_train, epochs=3, batch_size=4, verbose=0)
    # Evaluate the model on the validation set
    # You might want to split your data into a validation set beforehand.
    val_loss = model.evaluate(X_train, y_train, verbose=0, batch_size=4)
    model.reset_states()
    # Store the evaluation result
    evaluation_results[loss_name] = val_loss
    
    # Check if this loss function is the best so far
    if val_loss < best_loss_value:
        best_loss_value = val_loss
        best_loss_function = loss_name

print("Evaluation Results:")
for loss_name, val_loss in evaluation_results.items():
    print(f"{loss_name}: {val_loss}")

print(f"The best loss function is: {best_loss_function} with value: {best_loss_value}")

NameError: name 'loss_functions' is not defined